# Sequence to Sequence model for english to tamil transilation using rnn encoder and decoder ( at word level)

## dataset is downloaded from http://www.manythings.org/anki/

In [1]:
from __future__ import print_function

from keras.models import Model,load_model
from keras.layers import Input,LSTM,Dense
from keras.callbacks import ModelCheckpoint
import numpy as np

batch_size=128
epochs=25
latent_dim=256
data_path="/home/santhosh/keras/rnn/rnn_encoder_decoder/data/tam.txt"

/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## vectorizing the data

In [2]:
input_texts=[]
target_texts=[]
input_words=set()
target_words=set()
with open(data_path,'r',encoding='utf-8') as f:
    lines=f.read().split('\n')
for line in lines:
    line=line.split('\t')
    if(len(line)!=2):
        continue
    input_text,target_text=line
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text='<SOL> '+target_text+' <EOL>'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for word in input_text.split():
        if word not in input_words:
            input_words.add(word)
    for word in target_text.split():
        if word not in target_words:
            target_words.add(word)

In [3]:
input_words=sorted(list(input_words))
targer_words=sorted(list(target_words))
num_encoder_tokens=len(input_words)
num_decoder_tokens=len(target_words)
max_encoder_seq_len=max([len(text.split()) for text in input_texts])
max_decoder_seq_len=max([len(text.split()) for text in target_texts])

print('Number of samples:',len(input_texts))
print('number of unique input token:',num_encoder_tokens)
print('number of unique output token:',num_decoder_tokens)
print('Max Sequence length for inputs:',max_encoder_seq_len)
print('Max Sequence length for outputs:',max_decoder_seq_len)


Number of samples: 180
number of unique input token: 424
number of unique output token: 497
Max Sequence length for inputs: 19
Max Sequence length for outputs: 13


## defining token2index

In [4]:
input_token2index=dict([(word,i) for i,word in enumerate(input_words)])
target_token2index=dict([(word,i) for i,word in enumerate(target_words)])

## defing encoder_input,decoder_input and decoder_output

In [5]:
encoder_input_data=np.zeros((len(input_texts),max_encoder_seq_len,num_encoder_tokens),dtype='float32')
decoder_input_data=np.zeros((len(input_texts),max_decoder_seq_len,num_decoder_tokens),dtype='float32')
decoder_target_data=np.zeros((len(input_texts),max_decoder_seq_len,num_decoder_tokens),dtype='float32')


## creating training dataset

In [6]:
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t,word in enumerate(input_text.split()):
        encoder_input_data[i,t,input_token2index[word]]=1
        
    for t,word in enumerate(target_text.split()):
        decoder_input_data[i,t,target_token2index[word]]=1
        # decoder_target_data is ahead of decoder_input_data by one timestep
        if t>0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i,t-1,target_token2index[word]]=1


## Define an input sequence and process it.

In [7]:

encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]


## Set up the decoder, using encoder_states as initial state.

In [8]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

## Define the model that will turn
## `encoder_input_data` & `decoder_input_data` into `decoder_target_data`

In [9]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=["accuracy"])

## function to test model

In [10]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_word_index = dict(
    (i, word) for word, i in input_token2index.items())
reverse_target_word_index = dict(
    (i, word) for word, i in target_token2index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token2index['<SOL>']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_target_word_index[sampled_token_index]
        decoded_sentence += ' '+sampled_word

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_word == '<EOL>' or
           len(decoded_sentence.split()) > max_decoder_seq_len):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]
    if('<EOL>' in decoded_sentence):
        decoded_sentence=" ".join(decoded_sentence.split()[:-1])
    return decoded_sentence


## Run training

In [ ]:

iteration=0

# load weights
print('loading the weights')
model=load_model('word_level.h5')

# estimate accuracy on whole dataset using loaded weights
scores = model.evaluate([encoder_input_data, decoder_input_data], decoder_target_data,verbose=0)
print("%s: %.2f%%\n\n" % (model.metrics_names[1], scores[1]*100))
print("Testing Samples\n"+"-"*50)
for i in range(5):
    index=np.random.randint(len(input_texts))
    encoded_input_sequence=encoder_input_data[index: index + 1]
    output_sequence=decode_sequence(encoded_input_sequence)
    print(input_texts[index],output_sequence)
print("-"*50)

iteration_file="/home/santhosh/keras/rnn/rnn_encoder_decoder/iteration_word_level.txt"
try:
    file=open(iteration_file,'r')
    last_line=file.read().split('\n')[-2]
    print('file_data,',last_line)
    iteration=int(last_line.split(':')[1])
    #print(iteration)
    file.close()
    
except:
    print('no file exist')

# checkpoint
filepath="weights_best_word_level.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

while True:
    print('Iteration:',iteration+1)
    #training
    model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2,callbacks=callbacks_list)
    #prepare sample_data to test 5 samples:
    print("-"*50)
    for i in range(5):
        index=np.random.randint(len(input_texts))
        encoded_input_sequence=encoder_input_data[index: index + 1]
        output_sequence=decode_sequence(encoded_input_sequence)
        print(input_texts[index],output_sequence)
    print("-"*50)
    # Save model
    file=open(iteration_file,'a')
    file.write('iteration:'+str(iteration+1)+'\n')
    file.close()
    iteration+=1
    model.save('word_level.h5')

loading the weights
acc: 30.09%


Testing Samples
--------------------------------------------------
It's a piece of cake. இது ஒரு கேக்கின் துண்டு
Come and see me right now. உடனே வந்து என்னைப் பார்க்கவும்
When did you come to Japan? அவன் ஓட விருப்பப் படுகிறான்
Don't drink and drive. குடித்துவிட்டு வண்டி ஓட்டாதே
Do you have a lot of pens? உன்னிடம் நிறைய பேனாக்கள் இருக்கின்றனவா?
--------------------------------------------------
file_data, iteration:65
Iteration: 66
Train on 144 samples, validate on 36 samples
Epoch 1/25
144/144 [==============================] - 4s 25ms/step - loss: 2.2977e-05 - acc: 0.3542 - val_loss: 6.0619 - val_acc: 0.0876


/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/keras/engine/network.py:888: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1_2/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1_2/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


Epoch 2/25
144/144 [==============================] - 2s 12ms/step - loss: 2.2415e-05 - acc: 0.3542 - val_loss: 6.0665 - val_acc: 0.0876
Epoch 3/25
144/144 [==============================] - 2s 13ms/step - loss: 2.1443e-05 - acc: 0.3542 - val_loss: 6.0616 - val_acc: 0.0876
Epoch 4/25
144/144 [==============================] - 2s 12ms/step - loss: 2.1225e-05 - acc: 0.3542 - val_loss: 6.0646 - val_acc: 0.0876
Epoch 5/25
144/144 [==============================] - 2s 12ms/step - loss: 2.0248e-05 - acc: 0.3542 - val_loss: 6.0691 - val_acc: 0.0876
Epoch 6/25
144/144 [==============================] - 2s 13ms/step - loss: 1.9732e-05 - acc: 0.3542 - val_loss: 6.0735 - val_acc: 0.0876
Epoch 7/25
144/144 [==============================] - 2s 13ms/step - loss: 1.9039e-05 - acc: 0.3542 - val_loss: 6.0656 - val_acc: 0.0876
Epoch 8/25
144/144 [==============================] - 2s 13ms/step - loss: 1.8515e-05 - acc: 0.3542 - val_loss: 6.0769 - val_acc: 0.0876
Epoch 9/25
144/144 [=====================